# Factorial Function and Paramorphisms

In [1]:
from notebook_preamble import J, V, define

# `loop` Form

In [2]:
define('factorial == 1 swap dup 1 > [[*] dupdip -- dup 1 >] loop pop')

In [3]:
V('3 factorial')

                                  . 3 factorial
                                3 . factorial
                                3 . 1 swap dup 1 > [[*] dupdip -- dup 1 >] loop pop
                              3 1 . swap dup 1 > [[*] dupdip -- dup 1 >] loop pop
                              1 3 . dup 1 > [[*] dupdip -- dup 1 >] loop pop
                            1 3 3 . 1 > [[*] dupdip -- dup 1 >] loop pop
                          1 3 3 1 . > [[*] dupdip -- dup 1 >] loop pop
                         1 3 True . [[*] dupdip -- dup 1 >] loop pop
 1 3 True [[*] dupdip -- dup 1 >] . loop pop
                              1 3 . [*] dupdip -- dup 1 > [[*] dupdip -- dup 1 >] loop pop
                          1 3 [*] . dupdip -- dup 1 > [[*] dupdip -- dup 1 >] loop pop
                              1 3 . * 3 -- dup 1 > [[*] dupdip -- dup 1 >] loop pop
                                3 . 3 -- dup 1 > [[*] dupdip -- dup 1 >] loop pop
                              3 3 . -- dup 1 > [[*] dupdip --

In [4]:
define('P == dup 1 >')
define('factorial == 1 swap P [[*] dupdip -- P] loop pop')

In [5]:
J('3 factorial')

6


We have a form:

    1 swap P [[*] dupdip -- P] loop pop
    n swap P [[A] dupdip B  P] loop pop

With
- `n` is the "identity" for `A`
- `A :: (a, a) -> a`
- `B :: a -> a` generates the next value `a`
- and lastly `P :: a -> Bool` detects the end of the series.

The form starts with some value on the stack and duplicates it.  One copy is combined with the current value by `A`, and the other is modified by `B` to generate the next value for the loop, which continues until `P` returns `False`.

    n [A] [B] [P] paramorphism
    ...
    n swap P [[A] dupdip B P] loop pop



    n  [A] [B] [P] concat
    n  [A] [B P] [dupdip] swoncat
    n  [A] [dupdip B P] cons
    n [[A] dupdip B P]

Introduce the other `[P]`:

    n swap P [[A] dupdip B P]                        loop pop
    n        [[A] dupdip B P] [swap P]           dip loop pop
    n        [[A] dupdip B P] [P] [swap] swoncat dip loop pop

Putting them together:

    n [A] [B] [P] [concat [dupdip] swoncat cons] dipdup [swap] swoncat dip loop pop
    n [A] [B] [P]  concat [dupdip] swoncat cons [P] [swap] swoncat dip loop pop
    n [A] [B   P]         [dupdip] swoncat cons [P] [swap] swoncat dip loop pop
    n [A] [dupdip B P]                    cons [P] [swap] swoncat dip loop pop
    n [[A] dupdip B P] [P] [swap] swoncat dip loop pop
    n [[A] dupdip B P] [swap P]           dip loop pop
    n swap P [[A] dupdip B P] loop pop



In [6]:
define('paramorphism == [concat [dupdip] swoncat cons] dupdip [swap] swoncat dip loop pop')
define('factorial == 1 [*] [--] [P] paramorphism')

In [7]:
V('3 factorial')

                                                . 3 factorial
                                              3 . factorial
                                              3 . 1 [*] [--] [P] paramorphism
                                            3 1 . [*] [--] [P] paramorphism
                                        3 1 [*] . [--] [P] paramorphism
                                   3 1 [*] [--] . [P] paramorphism
                               3 1 [*] [--] [P] . paramorphism
                               3 1 [*] [--] [P] . [concat [dupdip] swoncat cons] dupdip [swap] swoncat dip loop pop
3 1 [*] [--] [P] [concat [dupdip] swoncat cons] . dupdip [swap] swoncat dip loop pop
                               3 1 [*] [--] [P] . concat [dupdip] swoncat cons [P] [swap] swoncat dip loop pop
                                 3 1 [*] [-- P] . [dupdip] swoncat cons [P] [swap] swoncat dip loop pop
                        3 1 [*] [-- P] [dupdip] . swoncat cons [P] [swap] swoncat dip loop pop
           

# `primrec` Recursive Form
    n swap [P] [pop] [[A] dupdip B] primrec
    
(Note that `P` is inverted from above, becoming `1 <=`.  Also it no longer requires `dup` because in the `primrec` combinator it is executed with its own copy of the stack so it can consume items so long as it leaves a Boolean on top when it's finished.)

In [8]:
define('factorial == 1 swap [1 <=] [pop] [[*] dupdip --] primrec')

In [9]:
V('3 factorial')

                                                                                   . 3 factorial
                                                                                 3 . factorial
                                                                                 3 . 1 swap [1 <=] [pop] [[*] dupdip --] primrec
                                                                               3 1 . swap [1 <=] [pop] [[*] dupdip --] primrec
                                                                               1 3 . [1 <=] [pop] [[*] dupdip --] primrec
                                                                        1 3 [1 <=] . [pop] [[*] dupdip --] primrec
                                                                  1 3 [1 <=] [pop] . [[*] dupdip --] primrec
                                                  1 3 [1 <=] [pop] [[*] dupdip --] . primrec
                                                  1 3 [1 <=] [pop] [[*] dupdip --] . [i] genrec
                 

### Derive `paramorphism` form the form above.
    n swap [P]       [pop]     [[A] dupdip B]                  primrec
    n [P] [swap] dip [pop]     [[A] dupdip B]                  primrec
    n [P] [[A] dupdip B]                [[swap] dip [pop]] dip primrec
    n [P] [A] [dupdip B]           cons [[swap] dip [pop]] dip primrec
    n [P] [A] [B] [dupdip] swoncat cons [[swap] dip [pop]] dip primrec

(Note the order of arguments is different, `[P]` is no longer at the top of the stack.)

    paramorphism == [dupdip] swoncat cons [[swap] dip [pop]] dip primrec

In [10]:
define('paramorphism == [dupdip] swoncat cons [[swap] dip [pop]] dip primrec')
define('factorial == 1 [1 <=] [*] [--] paramorphism')

In [11]:
V('3 factorial')

                                                                                   . 3 factorial
                                                                                 3 . factorial
                                                                                 3 . 1 [1 <=] [*] [--] paramorphism
                                                                               3 1 . [1 <=] [*] [--] paramorphism
                                                                        3 1 [1 <=] . [*] [--] paramorphism
                                                                    3 1 [1 <=] [*] . [--] paramorphism
                                                               3 1 [1 <=] [*] [--] . paramorphism
                                                               3 1 [1 <=] [*] [--] . [dupdip] swoncat cons [[swap] dip [pop]] dip primrec
                                                      3 1 [1 <=] [*] [--] [dupdip] . swoncat cons [[swap] dip [pop]] dip primrec
   

# `tails`
An example of a paramorphism for lists given in the ["Bananas..." paper](http://citeseerx.ist.psu.edu/viewdoc/summary?doi=10.1.1.41.125) is `tails` which returns the list of "tails" of a list.

    [1 2 3] tails == [[] [3] [2 3]]
    
Using `paramorphism` we would write:

    tails == [] [not] [rest swons] [rest] paramorphism

In [12]:
define('tails == [] [not] [rest swons] [rest] paramorphism')

In [13]:
J('[1 2 3] tails')

[[] [3] [2 3]]


### Factoring `rest`
Right before the recursion begins we have:
    
    [] [1 2 3] [not] [pop] [[rest swons] dupdip rest] primrec
    
But we might prefer to factor `rest` in the quote:

    [] [1 2 3] [not] [pop] [rest [swons] dupdip] primrec

There's no way to do that with the `paramorphism` combinator as defined.  We would have to write and use a slightly different recursion combinator or just write it manually.  This is yet another place where the *sufficiently smart compiler* will one day automatically refactor the code.

In [14]:
J('[] [1 2 3] [not] [pop] [rest [swons] dupdip] primrec')

[[] [3] [2 3]]
